In [2]:
import pandas as pd

df = pd.read_csv('gs://rob_buck/loan.csv')
df.head()

/var/tmp/ipykernel_4388/2369202826.py:3: DtypeWarning: Columns (19,47,55,112,123,124,125,128,129,130,133,139,140,141) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('gs://rob_buck/loan.csv')


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,NaN,NaN,2500,2500,2500.0,36 months,13.56,84.92,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,30000,30000,30000.0,60 months,18.94,777.23,D,D2,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,5000,5000,5000.0,36 months,17.97,180.69,D,D1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,4000,4000,4000.0,36 months,18.94,146.51,D,D2,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,30000,30000,30000.0,60 months,16.14,731.78,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df.shape

(2260668, 145)

In [4]:
df['is_default'] = df['loan_status'].map({
    'Fully Paid': 0,
    'Current': 0,
    'In Grace Period': 0,
    'Does not meet the credit policy. Status:Fully Paid': 0,
    
    'Charged Off': 1,
    'Default': 1,
    'Late (31-120 days)': 1,
    'Late (16-30 days)': 1,
    'Does not meet the credit policy. Status:Charged Off': 1
})

In [5]:
key_features = [
    'loan_amnt',           # Loan amount
    'int_rate',            # Interest rate  
    'grade',               # LC grade (A-G)
    'annual_inc',          # Annual income
    'dti',                 # Debt-to-income ratio
    'delinq_2yrs',         # Delinquencies in past 2 years
    'revol_util',          # Credit utilization
    'emp_length',          # Employment length
    'home_ownership',      # Home ownership status
    'purpose',             # Loan purpose
    'is_default'            
]

In [6]:
df = df[key_features]

In [7]:
df.shape

(2260668, 11)

In [8]:
df.isnull().sum() 

loan_amnt              0
int_rate               0
grade                  0
annual_inc             4
dti                 1711
delinq_2yrs           29
revol_util          1802
emp_length        146907
home_ownership         0
purpose                0
is_default             0
dtype: int64

In [9]:
df['annual_inc'] = df['annual_inc'].fillna(df['annual_inc'].mean())
df['dti'] = df['dti'].fillna(df['dti'].median())
df['delinq_2yrs'] = df['delinq_2yrs'].fillna(df['delinq_2yrs'].median())
df['revol_util'] = df['revol_util'].fillna(df['revol_util'].median())
df['emp_length'] = df['emp_length'].fillna('Unknown')

In [10]:
df.isnull().sum() 

loan_amnt         0
int_rate          0
grade             0
annual_inc        0
dti               0
delinq_2yrs       0
revol_util        0
emp_length        0
home_ownership    0
purpose           0
is_default        0
dtype: int64

In [11]:
from sklearn.preprocessing import LabelEncoder

# OneHot encode categorical columns directly
for col in ['grade', 'emp_length', 'home_ownership', 'purpose']:
   encoder = LabelEncoder()
   df[col] = encoder.fit_transform(df[col])

df.head()

,loan_amnt,int_rate,grade,annual_inc,dti,delinq_2yrs,revol_util,emp_length,home_ownership,purpose,is_default
0,2500,13.56,2,55000.0,18.24,0.0,10.3,1,5,2,0
1,30000,18.94,3,90000.0,26.52,0.0,24.2,1,1,2,0
2,5000,17.97,3,59280.0,10.51,0.0,19.1,6,1,2,0
3,4000,18.94,3,92000.0,16.74,0.0,78.1,1,1,2,0
4,30000,16.14,2,57250.0,26.35,0.0,3.6,1,1,2,0


In [12]:
df.columns

Index(['loan_amnt', 'int_rate', 'grade', 'annual_inc', 'dti', 'delinq_2yrs',
       'revol_util', 'emp_length', 'home_ownership', 'purpose', 'is_default'],
      dtype='object')

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.drop('is_default', axis=1), df['is_default'], test_size=0.2, random_state=42)

In [14]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [15]:
print(f"X_train_scaled shape: {X_train_scaled.shape}")
print(f"X_test_scaled shape: {X_test_scaled.shape}")

X_train_scaled shape: (1808534, 10)
X_test_scaled shape: (452134, 10)


In [16]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
from sklearn.ensemble import RandomForestClassifier

param_dist = {
    'n_estimators': randint(100, 500),
    'max_depth': [10, 20, 30, 40, 50, None],
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 10),
    'max_features': ['auto', 'sqrt', 'log2']
}

model = RandomForestClassifier(random_state=42)

random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_dist,
    n_iter=30,  
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1,
    random_state=42
)

In [ ]:
random_search.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


/opt/conda/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


In [ ]:
print("Best parameters:", random_search.best_params_)
print("Best cross-validation score:", random_search.best_score_)

In [ ]:
model = random_search.best_estimator_

In [ ]:
pred = model.predict_proba(X_test_scaled)[: , 1]

In [ ]:
print(pred)